## Notebook 2.2 Data Understanding and Preprocessing of Support Tables

For all intents and purposes, this should be considered as the second real notebook that is part of the thesis work. In it, we will look at the support tables that are part of the original database. These tables have information concerning courses and student performance - meaningful features for our project.

#### 1. We are familiarized with the logs

Before going further, we should assess the remaining tables presented in the database. 

Recall, **logs record interactions with the system and we are looking for ways to determine whether these interactions can assist educators identify at risk students and high performing students.**

Thus, to make the best out of the logs, we will need to perform different segmentations and it is likely that we will need perform some filtering. 

### To do that, we will take a look at all tables

We will look at all tables and all columns to make a preliminary assessment of the utility of the available elements.
In general, these are support elements that will be used sparsely, as most of the relevant information is present in the logs.

The observation of each table will resort to the same chain of commands:

info -> to observe count and datatype of each column, 
describe -> a command that that returns the most notable descriptive statistics of each column.
The obeservation of each table ends with a look at the raw data (At least the visible rows).

#### 2. We'll start this notebook by importing all relevant packages and data

All data is stored in the csv files that were exported in the previous notebook. 

In order to minimize unecessary steps, as we import these csv files we will immediatly remove, from each dataset:
1. The first unnamed column,
2. All columns that are entirely made of missing values - we have detected some.
3. All numerical columns that are immediatly recognied as categorical (or likely to be categorical values) are also immediatly declared as categoricals - this does not mean that, upon further assessment, other features may be converted to objects,
4. All features that display no null values and have a single value are promptly removed as well, 
5. Features related to time are converted to appropriate format - this is ultimately an ad-hoc assessment, but an important one to make.

In [1]:
#import libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import warnings
warnings.filterwarnings('ignore')

In [2]:
#other tables with support information
context_table = pd.read_csv('../Data/R_Gonz_data_mdl_context.csv', #context table -> unclear utility
                           dtype = {
                                   'id': object,
                                   'contextlevel': object,
                                   'instanceid': object,
                                   'path': object,
                                   'depth': object,                       
                                   },).drop('Unnamed: 0', axis = 1).dropna(how = 'all', axis = 1)

course_table = pd.read_csv('../Data/R_Gonz_data_mdl_course.csv', #course table -> unclear utility
                           dtype = {
                                   'id': object,
                                   'category': object,
                                   'outcomeid': object,
                                   'summaryformat': object,
                                   'showgrades': object,
                                   'newsitems': object,
                                   'legacyfiles': object,
                                   'marker': object,
                                   'showreports': object,          
                                   'visible': object,
                                   'visibleold': object,
                                   'groupmode': object,
                                   'groupmodeforce': object,
                                   'defaultgroupingid': object,
                                   'lang': object,          
                                   'requested': object,
                                   'enablecompletion': object,
                                   'completionnotify': object,                              
                                   },).drop('Unnamed: 0', axis = 1).dropna(how = 'all', axis = 1) 

course_mod_table = pd.read_csv('../Data/R_Gonz_data_mdl_course_modules.csv', #course module table -> unclear utility
                           dtype = {
                                   'id': object,
                                   'module': object,
                                   'instance': object,
                                   'course': object,
                                   'section': object,
                                   'idnumber': object,
                                   'indent': object,
                                   'visible': object,
                                   'visibleold': object,
                                   'groupmode': object,
                                   'groupingin': object,
                                   'groupmembersonly': object,
                                   'visibleold': object,
                                   'groupmode': object,
                                   'groupingin': object,
                                   'groupmembersonly': object,
                                   'completion': object,
                                   'completionview': object,
                                   'showavailability': object,
                                   'showdescription': object,
                                   },).drop('Unnamed: 0', axis = 1).dropna(how = 'all', axis = 1) 

grades_table = pd.read_csv('../Data/R_Gonz_data_mdl_grade_grades.csv',  # grade table -> unclear utility
                           dtype = {
                                   'id': object,
                                   'itemid': object,
                                   'userid': object,
                                   'usermodified': object,
                                   'rawscaleid': object,
                                    'hidden': object,
                                   'feedback': object,
                                   'userid': object,
                                   'feedbackformat': object,
                                   'informationformat': object, 
                                   },).drop('Unnamed: 0', axis = 1).dropna(how = 'all', axis = 1)

grade_item_table = pd.read_csv('../Data/R_Gonz_data_mdl_grade_items.csv', # grade_items table -> unclear utility
                           dtype = {
                                   'id': object,
                                   'itemid': object,
                                   'categoryid': object,
                                   'courseid': object,
                                   'idnumber': object,
                                   'iteminstance' : object,
                                   'itemnumber' : object,
                                   'gradetype': object,
                                   'scaleid': object,
                                   'multfactor': object,
                                   'outcomeid': object,                          
                                   },).drop('Unnamed: 0', axis = 1).dropna(how = 'all', axis = 1) 

role_assign_table = pd.read_csv('../Data/R_Gonz_data_mdl_role_assignments.csv', # role assignments table -> unclear utility
                           dtype = {
                                   'id': object,
                                   'roleid': object,
                                   'contextid': object,
                                   'itemid': object,
                                   'userid': object,
                                   'modifierid': object,
                                   },).drop('Unnamed: 0', axis = 1).dropna(how = 'all', axis = 1)

#### First, the role assignment tables

The role assignment table is the table where the user assignments of the database are present.
With it, we can see which roles exist and, ultimately, filter for their role - which is represented in this table by the column roleid.

In [3]:
role_assign_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219297 entries, 0 to 219296
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            219297 non-null  object 
 1   roleid        219297 non-null  object 
 2   contextid     219297 non-null  object 
 3   userid        219297 non-null  object 
 4   timemodified  219297 non-null  float64
 5   modifierid    219297 non-null  object 
 6   component     194425 non-null  object 
 7   itemid        219297 non-null  object 
 8   sortorder     219297 non-null  float64
dtypes: float64(2), object(7)
memory usage: 15.1+ MB


In [4]:
role_assign_table.drop([
                    'sortorder',
                    ],
                    axis = 1, inplace = True)

#timemodifieds seem to be a time feature, so we will appropriately make the conversion to datetime
role_assign_table['timemodified'] = pd.to_datetime(role_assign_table['timemodified'], unit = 's', errors = 'coerce')

In [5]:
role_assign_table['roleid'].value_counts()

5.0    208550
3.0     10743
4.0         4
Name: roleid, dtype: int64

We see that this database has 30813 unique users and 3 unique roles. As the data that is part of this database deals with information collected throughout one school-year, it is likely that the most represented role is the role of student.

we will find the most common value in the role in role id and filter the role assignment table, only keeping rows where that role is present. 

In [6]:
#get most common role and filter approppriately
student_role = list(role_assign_table['roleid'].mode())

#we will create 2 dataframes - one with the students and another with other members
other_roles_tables = role_assign_table[~(role_assign_table['roleid'].isin(student_role))]
role_assign_table = role_assign_table[role_assign_table['roleid'].isin(student_role)]

#we will also create a list will all users whose role is student
students = role_assign_table['userid'].unique()

In [7]:
role_assign_table.describe(include = 'all', datetime_is_numeric = True).T

,count,unique,top,freq,mean,min,25%,50%,75%,max
id,208550,208550,2.0,1,NaT,NaT,NaT,NaT,NaT,NaT
roleid,208550,1,5.0,208550,NaT,NaT,NaT,NaT,NaT,NaT
contextid,208550,4215,229861.0,24826,NaT,NaT,NaT,NaT,NaT,NaT
userid,208550,29062,38881.0,33,NaT,NaT,NaT,NaT,NaT,NaT
timemodified,208550,NaN,NaN,NaN,2014-09-21 10:15:55.523346944,2014-07-01 11:13:21,2014-08-05 22:34:00,2014-09-03 03:15:18,2014-10-15 18:15:47.750000128,2015-07-30 23:41:19
modifierid,208550,27,0.0,208416,NaT,NaT,NaT,NaT,NaT,NaT
component,183687,1,enrol_database,183687,NaT,NaT,NaT,NaT,NaT,NaT
itemid,208550,4208,0.0,24863,NaT,NaT,NaT,NaT,NaT,NaT


In [8]:
#use this cell to write any additional piece of code that may be required

**Next, we'll consider grades**

Student performance is, in general, measured by the student's grade. So... how do we measure grades?
As all we have is data from Moodle, it is important that we can either find or calculate the targets from Moodle data.

So, in an immediate fashion, we'll have to identify which courses have graded assignments and slice a course list for those.
we can deal with calculating our target at a later stage.

In [9]:
grades_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437650 entries, 0 to 437649
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 437650 non-null  object 
 1   itemid             437650 non-null  object 
 2   userid             437650 non-null  object 
 3   rawgrade           137820 non-null  float64
 4   rawgrademax        437650 non-null  float64
 5   rawgrademin        437650 non-null  float64
 6   rawscaleid         88518 non-null   object 
 7   usermodified       437650 non-null  object 
 8   finalgrade         236668 non-null  float64
 9   hidden             437650 non-null  object 
 10  locked             437650 non-null  float64
 11  locktime           437650 non-null  float64
 12  exported           437650 non-null  float64
 13  overridden         437650 non-null  float64
 14  excluded           437650 non-null  float64
 15  feedback           20273 non-null   object 
 16  fe

In [10]:
#the informationformat feature has no null values and is a single value feature, so we remove it
grades_table.drop('informationformat', axis = 1, inplace = True)

#time created and timemodified seem to be time features, so we will appropriately make the conversion to datetime
grades_table['timecreated'] = pd.to_datetime(grades_table['timecreated'], unit = 's', errors = 'coerce')
grades_table['timemodified'] = pd.to_datetime(grades_table['timemodified'], unit = 's', errors = 'coerce')

#experimental - to delete if need be - only keeping students - assuming we have correctly only kept students
#grades_table = grades_table[grades_table['userid'].isin(students)]

In [11]:
grades_table.describe(include ='all', datetime_is_numeric = True).T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
id,437650,437650,130.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
itemid,437650,12474,12132.0,671,NaN,NaN,NaN,NaN,NaN,NaN,NaN
userid,437650,18015,71424.0,170,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rawgrade,137820.0,NaN,NaN,NaN,49.942597,-11.73913,4.772815,10.0,60.0,1001.0,120.226285
rawgrademax,437650.0,NaN,NaN,NaN,88.676712,0.0,10.0,100.0,100.0,1001.0,179.980738
rawgrademin,437650.0,NaN,NaN,NaN,0.206957,-1.0,0.0,0.0,0.0,5.0,0.40969
rawscaleid,88518,186,4.0,32578,NaN,NaN,NaN,NaN,NaN,NaN,NaN
usermodified,437650,15357,0.0,62427,NaN,NaN,NaN,NaN,NaN,NaN,NaN
finalgrade,236668.0,NaN,NaN,NaN,43.902587,0.0,2.45,8.05,51.0,1140.8,112.796347
hidden,437650,2,0.0,417028,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#use this cell to write any additional piece of code that may be required

**Next, we have the Grade_item_table**

The grade_item table stores information concerning every gradeable item present in the database.

In [13]:
grade_item_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30028 entries, 0 to 30027
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               30028 non-null  object 
 1   courseid         30028 non-null  object 
 2   categoryid       23322 non-null  object 
 3   itemname         30028 non-null  object 
 4   itemtype         30028 non-null  object 
 5   itemmodule       21372 non-null  object 
 6   iteminstance     30028 non-null  object 
 7   itemnumber       23322 non-null  object 
 8   idnumber         3454 non-null   object 
 9   calculation      424 non-null    object 
 10  gradetype        30028 non-null  object 
 11  grademax         30028 non-null  float64
 12  grademin         30028 non-null  float64
 13  scaleid          4768 non-null   object 
 14  outcomeid        25260 non-null  object 
 15  gradepass        30028 non-null  float64
 16  multfactor       30028 non-null  object 
 17  plusfactor  

In [14]:
grade_item_table['gradetype'].value_counts()

1.0    23935
2.0     4783
3.0     1205
0.0      105
Name: gradetype, dtype: int64

In [15]:
#the informationformat feature has no null values and is a single value feature, so we remove it
grade_item_table.drop([
                    'itemname',
                    'plusfactor',
                    'timecreated',
                    'timemodified'
                    ],
                    axis = 1, inplace = True)

grade_item_table.rename(columns = {'id' : 'itemid'}, inplace = True)

In [16]:
grade_item_table.describe(include ='all', datetime_is_numeric = True).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
itemid,30028,30028,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
courseid,30028,5551,1073.0,146,NaN,NaN,NaN,NaN,NaN,NaN,NaN
categoryid,23322,3363,1191.0,145,NaN,NaN,NaN,NaN,NaN,NaN,NaN
itemtype,30028,4,mod,21372,NaN,NaN,NaN,NaN,NaN,NaN,NaN
itemmodule,21372,11,assign,15773,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iteminstance,30028,17307,0.0,1950,NaN,NaN,NaN,NaN,NaN,NaN,NaN
itemnumber,23322,1,0.0,23322,NaN,NaN,NaN,NaN,NaN,NaN,NaN
idnumber,3454,2691,1,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
calculation,424,419,=101+102,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gradetype,30028,4,1.0,23935,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
grade_item_table['itemmodule'].value_counts()

assign           15773
quiz              4517
forum              264
scorm              237
questionnaire      214
workshop           195
glossary            91
lesson              47
nanogong            21
data                12
pcast                1
Name: itemmodule, dtype: int64

In [18]:
#use this cell to write any additional piece of code that may be required

#### It is possible to see that only a subset of courses has graded items.

As performed by the authors of the Riestra-Gonzalez paper, we will only look to work with courses that have graded assignments. 
The reason for this option is straightforward - we have no access to the SIS, which means that our target will be, in some shape or form, related to the graded assignments.

**After looking at the grades tables, it is important to incorporate the information presented in these tables with the tables about courses**.

For that, we have access to multiple dfs related to the courses themselves.
Behold, the course_table.

In [19]:
course_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5732 entries, 0 to 5731
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5732 non-null   object 
 1   category           5732 non-null   object 
 2   sortorder          5732 non-null   float64
 3   fullname           5732 non-null   object 
 4   shortname          5732 non-null   object 
 5   idnumber           5731 non-null   object 
 6   summary            5732 non-null   object 
 7   summaryformat      5732 non-null   object 
 8   format             5732 non-null   object 
 9   showgrades         5732 non-null   object 
 10  newsitems          5732 non-null   object 
 11  startdate          5732 non-null   float64
 12  marker             5732 non-null   object 
 13  maxbytes           5732 non-null   float64
 14  legacyfiles        5732 non-null   object 
 15  showreports        5732 non-null   object 
 16  visible            5732 

In [20]:
course_table.describe(include ='all', datetime_is_numeric = True).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,5732,5732,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
category,5732,74,1.0,3589,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sortorder,5732.0,NaN,NaN,NaN,213321.98866,313262.684429,1.0,11432.75,12865.5,380005.25,850116.0
fullname,5732,1,nombre,5732,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shortname,5732,5732,Uniovi Virtual,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
idnumber,5731,5731,"T_1C,A_15473",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
summary,5732,1,-,5732,NaN,NaN,NaN,NaN,NaN,NaN,NaN
summaryformat,5732,3,1.0,3316,NaN,NaN,NaN,NaN,NaN,NaN,NaN
format,5732,5,topics,5710,NaN,NaN,NaN,NaN,NaN,NaN,NaN
showgrades,5732,2,1.0,5228,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#the course_table has multiple single value feature with no nans, as with the previous considered features, we will remove them

course_table.drop(['fullname',
                       'summary',
                       'requested',
                       'enablecompletion',
                       'completionnotify'],
                       axis = 1, inplace = True)

#time created and timemodified seem to be time features, so we will appropriately make the conversion to datetime
course_table['startdate'] = pd.to_datetime(course_table['startdate'], unit = 's', errors = 'coerce')
course_table['timecreated'] = pd.to_datetime(course_table['timecreated'], unit = 's', errors = 'coerce')
course_table['timemodified'] = pd.to_datetime(course_table['timemodified'], unit = 's', errors = 'coerce')
course_table['cacherev'] = pd.to_datetime(course_table['cacherev'], unit = 's', errors = 'coerce') 

In [22]:
course_table

,id,category,sortorder,shortname,idnumber,summaryformat,format,showgrades,newsitems,startdate,...,showreports,visible,visibleold,groupmode,groupmodeforce,defaultgroupingid,lang,timecreated,timemodified,cacherev
0,1.0,0.0,1.0,Uniovi Virtual,NaN,0.0,site,1.0,3.0,1970-01-01 00:00:00,...,0.0,1.0,1.0,0.0,0.0,0.0,NaN,2014-06-05 17:08:12,2014-08-25 13:13:00,2015-06-03 12:11:01
1,3.0,1.0,13589.0,"T_1C,A_15473","T_1C,A_15473",1.0,topics,1.0,1.0,1999-11-29 23:00:00,...,0.0,0.0,1.0,0.0,0.0,0.0,NaN,2011-06-06 14:32:36,2011-09-12 12:25:12,2015-03-02 16:33:13
2,4.0,1.0,13588.0,"T_1C,A_15470","T_1C,A_15470",1.0,topics,1.0,1.0,2012-10-02 22:00:00,...,0.0,0.0,1.0,0.0,0.0,0.0,NaN,2011-09-26 09:11:45,2012-10-03 01:45:30,2015-03-02 16:33:13
3,5.0,1.0,13587.0,"T_1C,A_15181","T_1C,A_15181",1.0,topics,1.0,1.0,2014-01-21 23:00:00,...,0.0,0.0,1.0,0.0,0.0,0.0,NaN,2011-06-06 14:32:51,2014-01-22 18:42:58,2015-03-02 16:33:13
4,6.0,61.0,310007.0,"T_S,A_MGENYDIV-1-022","T_S,A_MGENYDIV-1-022",1.0,topics,1.0,1.0,2015-01-31 23:00:00,...,0.0,1.0,1.0,0.0,0.0,0.0,NaN,2011-06-20 10:49:05,2015-02-01 21:32:00,2015-03-02 16:33:13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5727,5928.0,108.0,790005.0,"T_CL,P_10119,A_6634","T_CL,P_10119,A_6634",0.0,topics,1.0,5.0,1970-01-01 00:00:00,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2015-07-01 09:17:43,2015-07-01 09:17:43,2015-07-01 09:17:43
5728,5929.0,108.0,790004.0,"T_CL,P_10119,A_6635","T_CL,P_10119,A_6635",0.0,topics,1.0,5.0,1970-01-01 00:00:00,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2015-07-01 09:17:44,2015-07-01 09:17:44,2015-07-06 12:39:03
5729,5930.0,108.0,790003.0,"T_CL,P_10119,A_6624","T_CL,P_10119,A_6624",0.0,topics,1.0,5.0,1970-01-01 00:00:00,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2015-07-01 09:17:44,2015-07-01 09:17:44,2015-07-01 09:17:44
5730,5931.0,108.0,790002.0,"T_CL,P_10119,A_6625","T_CL,P_10119,A_6625",0.0,topics,1.0,5.0,1970-01-01 00:00:00,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2015-07-01 09:17:45,2015-07-01 09:17:45,2015-07-06 12:39:02


In [23]:
#use this cell to write any additional piece of code that may be required

#### The course module table is present in other datasets "e.g. The Open Moodle Dataset", 

According to it, the course module table describes every activity performed with Moodle. In our case, it records every activity performed in every course.

Here follows a brief overview of this table.

In [24]:
course_mod_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228216 entries, 0 to 228215
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  228216 non-null  object 
 1   course              228216 non-null  object 
 2   module              228216 non-null  object 
 3   instance            228216 non-null  object 
 4   section             228216 non-null  object 
 5   idnumber            428 non-null     object 
 6   added               228216 non-null  float64
 7   score               228216 non-null  float64
 8   indent              228216 non-null  object 
 9   visible             228216 non-null  object 
 10  visibleold          228216 non-null  object 
 11  groupmode           228216 non-null  object 
 12  groupingid          228216 non-null  float64
 13  groupmembersonly    228216 non-null  object 
 14  completion          228216 non-null  object 
 15  completionview      228216 non-nul

In [25]:
#the course_table has multiple single value feature with no nans, as with the previous considered features, we will remove them

course_mod_table.drop([
                    'groupmembersonly',
                    'completion',
                    'completionview',
                    'showdescription',
                    'completionexpected',
                    'score',
                    ],
                    axis = 1, inplace = True)

#added, availablefrom and availableuntil seem to be time features, so we will appropriately make the conversion to datetime
course_mod_table['added'] = pd.to_datetime(course_mod_table['added'], unit = 's', errors = 'coerce')
course_mod_table['availablefrom'] = pd.to_datetime(course_mod_table['availablefrom'], unit = 's', errors = 'coerce')
course_mod_table['availableuntil'] = pd.to_datetime(course_mod_table['availableuntil'], unit = 's', errors = 'coerce')

#renaming variables that we will use later on for mergers
course_mod_table.rename(columns = {'instance': 'iteminstance', 'course': 'courseid', 'id' : 'assign_id'}, inplace = True)

In [26]:
course_mod_table.describe(include = 'all', datetime_is_numeric = True).T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
assign_id,228216,228216,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
courseid,228216,5643,4460.0,575,NaN,NaN,NaN,NaN,NaN,NaN,NaN
module,228216,23,17.0,144178,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iteminstance,228216,150172,10.0,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
section,228216,40573,51699.0,268,NaN,NaN,NaN,NaN,NaN,NaN,NaN
idnumber,428,288,2,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
added,228216,NaN,NaN,NaN,2012-08-19 22:31:16.422490368,2006-02-13 09:09:40,2011-06-01 18:43:16.500000,2013-02-28 00:00:26.500000,2014-04-30 08:43:07.750000128,2015-07-30 14:00:40,NaN
indent,228216,34,0.0,166823,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visible,228216,2,1.0,173801,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visibleold,228216,2,1.0,199037,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
course_mod_table

,assign_id,courseid,module,iteminstance,section,idnumber,added,indent,visible,visibleold,groupmode,groupingid,availablefrom,availableuntil,showavailability
0,1.0,1.0,25.0,1.0,1.0,NaN,2014-06-06 08:03:57,0.0,1.0,0.0,0.0,0.0,1970-01-01,1970-01-01,0.0
1,2.0,3.0,9.0,1.0,3.0,NaN,2008-07-28 08:33:14,0.0,0.0,1.0,0.0,0.0,1970-01-01,1970-01-01,0.0
2,3.0,3.0,12.0,1.0,3.0,NaN,2012-01-09 11:55:42,0.0,1.0,1.0,0.0,0.0,1970-01-01,1970-01-01,0.0
3,4.0,3.0,17.0,1.0,3.0,NaN,2012-01-25 17:43:03,0.0,1.0,1.0,0.0,0.0,1970-01-01,1970-01-01,0.0
4,5.0,3.0,17.0,2.0,3.0,NaN,2012-01-25 18:00:11,0.0,1.0,1.0,0.0,0.0,1970-01-01,1970-01-01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228211,276295.0,4721.0,17.0,179175.0,63006.0,NaN,2015-07-29 18:07:15,0.0,1.0,1.0,0.0,0.0,1970-01-01,1970-01-01,1.0
228212,276298.0,4721.0,20.0,17945.0,63006.0,NaN,2015-07-30 00:28:31,0.0,1.0,1.0,0.0,0.0,1970-01-01,1970-01-01,1.0
228213,276299.0,4721.0,20.0,17946.0,63006.0,NaN,2015-07-30 00:30:30,0.0,1.0,1.0,0.0,0.0,1970-01-01,1970-01-01,1.0
228214,276305.0,4448.0,9.0,9557.0,55307.0,NaN,2015-07-30 10:51:02,0.0,1.0,1.0,0.0,0.0,1970-01-01,1970-01-01,0.0


In [28]:
#use this cell to write any additional piece of code that may be required

The last set of tables to check is the one that contains the context_table. The utility of these tables is rather unclear at this moment.

#### Context table

In [29]:
context_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351126 entries, 0 to 351125
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            351126 non-null  object
 1   contextlevel  351126 non-null  object
 2   instanceid    351126 non-null  object
 3   path          351126 non-null  object
 4   depth         351126 non-null  object
dtypes: object(5)
memory usage: 13.4+ MB


In [30]:
context_table.describe(include = 'all', datetime_is_numeric = True).T

,count,unique,top,freq
id,351126,351126,1.0,1
contextlevel,351126,6,70.0,228216
instanceid,351126,240746,119.0,5
path,351126,351126,/1,1
depth,351126,6,4.0,224241


In [31]:
context_table

,id,contextlevel,instanceid,path,depth
0,1.0,10.0,0.0,/1,1.0
1,4.0,50.0,1.0,/1/4,2.0
2,6.0,30.0,1.0,/1/6,2.0
3,7.0,30.0,2.0,/1/7,2.0
4,8.0,80.0,1.0,/1/4/8,3.0
...,...,...,...,...,...
351121,400993.0,30.0,81852.0,/1/400993,2.0
351122,400994.0,30.0,81853.0,/1/400994,2.0
351123,400995.0,30.0,81854.0,/1/400995,2.0
351124,400996.0,30.0,81855.0,/1/400996,2.0


In [32]:
#use this cell to write any additional piece of code that may be required

#### 3. To business

The information stored in these tables is pivotal for our work with the logs. Ignoring all other noise potential insights that may arise from this data we are, for the most part, interested in 3 things:

1. Identify the student population - already achieved
2. Compute Student Performance - our target
3. Get course duration - or find a way to compute those - the courses to that we will take forward.

We've been discussing continuously that we want to, in some capacity, predict student performance. As we do not have access to the final grades, we will need to infer it from graded Moodle assignments. The first, and almost immediate observation is that we will can only use courses that use Moodle in this capacity -> which will reduce the number of courses we have to work with.

We will follow the formula adopted by the authors of the Riestra-González paper:

#### Student Performance and Course Duration

The authors got to student performance and course duration by performing inner joins across multiple tables and filtered across different conditions:

course_mod_table,
grades_table,
grade_item_table

We will replicate their steps and hopefully, reach suport tables that return comparable results. The first step is to perform the removal of rows that will be unnecessary for us. We can only construct a solution for items that are graded and for which we have the means to estimate the course duration. 

Thus, in the grades_table, we will look to only keep rows that can, simultaneuously, fulfill the following pre-requisite:
1. Have a valid final grade,

The second phase will be to perform inner joins of the different tables:
1. course_mod_table with grade_item_table on iteminstance and courseid
2. grade_item_table.id with grades_table.itemid
3. The merge of the previous 2 merged tables

In [33]:
# #Step 1, removing all rows that have no interest to us

# grades_table.dropna(subset = ['finalgrade','timecreated', 'timemodified'], inplace = True)

#Step 2: Create temporary tables that associate courses and assignments
placeholder_1 = pd.merge(course_mod_table, grade_item_table, on=['iteminstance','courseid'], how='inner')

#Step 3: Create second temporary table that associates grades with assignments
placeholder_2 = pd.merge(placeholder_1, grades_table, on ='itemid', how='inner')

#step 3: merge both placeholder tables
support_table = placeholder_2[:]
support_table['sup_time'] = np.where(support_table['timecreated'] > support_table['timemodified'],
                                support_table['timecreated'], support_table['timemodified'])

#step 4: only keep graded items, which means nonzero max grades
support_table = support_table[(support_table['rawgrademax'] > 0) & (support_table['sup_time'].dt.year >= 2014)]

del placeholder_1, placeholder_2

**As a final step, we will store the start date of each course - as it will provide us with the means to, further down the line, perform the inference for course duration.**

In [34]:
#only keep rows worth merging - this cell can only be run once
course_table = course_table.filter(['id', 'startdate']).rename(columns = {'id': 'courseid'})

#perform inner join between support table and courses with grades
support_table = pd.merge(support_table, course_table, on = 'courseid', how = 'inner')

#only keep the final result
support_table = support_table[(support_table['startdate'] <= support_table['sup_time']) & (support_table['startdate'].dt.year >= 2014)].filter(['assign_id', 'courseid', 'startdate', 'userid', 'finalgrade', 
                                      'rawgrademax', 'sup_time'])

In [35]:
support_table.describe(include = 'all', datetime_is_numeric = True).T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
assign_id,271382,8742,98648.0,657,NaN,NaN,NaN,NaN,NaN,NaN,NaN
courseid,271382,1260,2271.0,17871,NaN,NaN,NaN,NaN,NaN,NaN,NaN
startdate,271382,NaN,NaN,NaN,2014-11-21 15:33:49.810378240,2014-07-01 00:00:00,2014-09-09 22:00:00,2014-10-28 23:00:00,2015-01-26 23:00:00,2015-07-04 22:00:00,NaN
userid,271382,16502,71424.0,140,NaN,NaN,NaN,NaN,NaN,NaN,NaN
finalgrade,136275.0,NaN,NaN,NaN,59.966919,0.0,5.0,10.0,65.0,1001.0,141.23242
rawgrademax,271382.0,NaN,NaN,NaN,105.566822,0.07,10.0,100.0,100.0,1001.0,215.809835
sup_time,271382,NaN,NaN,NaN,2015-02-04 13:33:52.513117952,2014-07-31 14:57:13,2014-11-28 22:00:31.500000,2015-02-06 17:59:00,2015-04-15 11:30:44,2015-07-30 19:17:37,NaN


We will finish this section by filtering the features to keep and, afterward, export the support table to use with the LMS logs. 

#### Section 2: 

By now, we know, generally:

- all courses that had graded assignments (i.e. whose max assignment grade was not 0) - courseid,
- all students that were registered in the curricular unit - userid,
- if a student delivered an assignment or not and the assignment's grade - finalgrade.

This information is especially useful because it will assist us in the proper filtering of the moodle activity logs and, additionally, assist us in the achievement of valuable information needed for the project: course duration and target.

#### 1. Course duration: 

We have the start date for each course. The authors of original paper inferred course end to occur at the 95% log threshold. That is to say, 5% of the logs were registered after the end of course. We have no way to obtain a better estimate so we will accept the postulation. When we deal with the logs, we will be able to calculate end of course date.

#### 2. Targets - finalgrade:

The authors calculated final grade as a construct computed from the assignment grades. Again, we will accept the author's methods for this. 

**First**: to classify whether different assignments were mandatory or not

The authors of the paper focused made a split between mandatory and optional assignments. In their view, any assignment whose submittal rate (relative to the number of students attending the course) is 40% or under would be considered an optional assignment. 

1. We will need to know which assignments are optional and which are mandatory. We have, from our support table, the ability to list the courses and students that attending the course. From here, we can get the number of students attending each course. 

In [36]:
# we get to create a pivot-table that associates students and the courses they are attending
student_list = pd.pivot_table(support_table, index='userid', columns = 'courseid', values = 'assign_id',
                    aggfunc='count')

# we use the describe command to get the course-level aggregate statistics
# count -> number of students attending, mean is the average number of clicks performed by each student 
student_count = student_list.describe(include = 'all').T.sort_values(by = 'count', ascending = False)['count'].reset_index()

#from here, we can create a dict that associates each course to the number of students attending the course
student_count = student_count.set_index('courseid').to_dict()['count']

2. We can, in some capacity, partially repeat the steps performed in the previous pivot-table and make the option/mandatory classification of each assignment.

In [37]:
# we get to create a pivot-table that associates assignments and the courses are asked on
assign_number = pd.pivot_table(support_table.dropna(), index= 'userid', columns = ['courseid', 'assign_id'], values = 'finalgrade',
                    aggfunc='count')

# we use the describe command to get the course-level aggregate statistics
# count -> number of students delivering the assignment, mean is the average number of students delivering the assignment 
assign_number = assign_number.describe(include = 'all').T.sort_values(by = 'count', ascending = False)['count'].reset_index()

#from her, we can create 2 columns: i) one with the number of students attending the course
assign_number['registered_students'] = assign_number['courseid'].map(student_count)

#then, we can calculate the percentage of assignments delivered relative to the number of attending students
assign_number['%_submissions'] = assign_number['count'] / assign_number['registered_students']

#finally, we classify each assignment as mandatory vs non-mandatory (over 40% submission rates)
assign_number['mandatory_status'] = np.where(assign_number['%_submissions'] > 0.4, 1, 0)

#from here, we can create a dict that associates each course to the number of students attending the course
mandatory_status = assign_number.set_index('assign_id').to_dict()['mandatory_status']

#from here, we can now map the mandatory vs non mandatory status of 
support_table['mandatory_status'] = support_table['assign_id'].map(mandatory_status)

del assign_number, student_count, mandatory_status

We now have assigned the mandatory status to different assignments. We will not use this knowledge immediatly, but we will need it later. What it allows us is the ability to perform new computations.

**3. Now, we can clean unnecessary assignments and courses. We can now perform the following operations:**

1. identify whether the students made the delivery of the assignment or not - nans vs non nans

2. give every nan the classification of 0.

3. verify whether any courses have average finalgrade of 0. By extension, every course that only has 0 mean finalgrades will also excluded.

4. Another variant we considered was the removal of all assignments with average finalgrade = 0. Ultimately, we opted to keep these in courses where there are assignments with finalgrade > 0,

5. Additionally, we considered the removal of average finalgrade equal to the rawgrademax . However, we opted to keep these records.

In [38]:
# check whether the assignment was delivered by the student or not
support_table['delivered'] = np.where(support_table['finalgrade'].isna(), 0, 1)

#now, we fill the nas of finalgrade with 0
support_table.fillna(0, inplace = True)

#as a final note, we can now verify which courses we can exclude
#criteria 1: if all assignments have average grade 0, the course can be excluded
assignments_keep = support_table.groupby(['courseid']).agg({
                                                    'userid': 'count',
                                                    'finalgrade' : 'mean',
                                                    'rawgrademax' : 'mean',
                                                    },
                                                    )

#now we select assignments that fulfill the criteria avg finalgrade = 0 and store it in a list
assignments_keep = list(assignments_keep[assignments_keep['finalgrade'] > 0].reset_index()['courseid'])

#so, we keep assignments who have a positive finalgrade
support_table = support_table[support_table['courseid'].isin(assignments_keep)]

del assignments_keep

#check
support_table

,assign_id,courseid,startdate,userid,finalgrade,rawgrademax,sup_time,mandatory_status,delivered
0,588.0,18.0,2014-09-03 22:00:00,17881.0,100.00000,100.0,2014-11-04 16:22:27,1.0,1
1,588.0,18.0,2014-09-03 22:00:00,18204.0,100.00000,100.0,2014-11-04 16:22:36,1.0,1
2,588.0,18.0,2014-09-03 22:00:00,18541.0,100.00000,100.0,2014-11-04 16:22:45,1.0,1
3,588.0,18.0,2014-09-03 22:00:00,22888.0,100.00000,100.0,2014-11-04 16:22:51,1.0,1
4,588.0,18.0,2014-09-03 22:00:00,23037.0,100.00000,100.0,2014-11-04 16:22:58,1.0,1
...,...,...,...,...,...,...,...,...,...
304114,275678.0,5909.0,2015-07-04 22:00:00,78349.0,59.21053,100.0,2015-07-28 20:06:44,0.0,1
304115,275678.0,5909.0,2015-07-04 22:00:00,78352.0,90.78947,100.0,2015-07-28 20:08:25,0.0,1
304116,275678.0,5909.0,2015-07-04 22:00:00,78439.0,35.52632,100.0,2015-07-28 20:11:42,0.0,1
304117,275678.0,5909.0,2015-07-04 22:00:00,78442.0,59.21053,100.0,2015-07-28 20:13:11,0.0,1


In [39]:
support_table.describe(include = 'all', datetime_is_numeric = True).T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
assign_id,221523,6347,98648.0,657,NaN,NaN,NaN,NaN,NaN,NaN,NaN
courseid,221523,732,2271.0,17871,NaN,NaN,NaN,NaN,NaN,NaN,NaN
startdate,221523,NaN,NaN,NaN,2014-11-24 06:10:46.405113600,2014-07-31 22:00:00,2014-09-09 22:00:00,2014-11-30 23:00:00,2015-01-26 23:00:00,2015-07-04 22:00:00,NaN
userid,221523,14121,68626.0,131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
finalgrade,221523.0,NaN,NaN,NaN,36.890038,0.0,0.0,2.5,25.0,1001.0,114.550804
rawgrademax,221523.0,NaN,NaN,NaN,107.643393,0.07,10.0,41.0,100.0,1001.0,233.260426
sup_time,221523,NaN,NaN,NaN,2015-02-09 13:41:47.534499840,2014-08-01 14:37:32,2014-12-05 10:45:46.500000,2015-02-13 12:45:39,2015-04-20 19:53:53.500000,2015-07-28 21:45:39,NaN
mandatory_status,221523.0,NaN,NaN,NaN,0.563689,0.0,0.0,1.0,1.0,1.0,0.495928
delivered,221523.0,NaN,NaN,NaN,0.614713,0.0,0.0,1.0,1.0,1.0,0.486664


**4. Before finishing this notebook, there is still one thing we need to do:**

So far, the students registered in each course, and their results in graded assignments.

The authors of the Riestra González paper used the following equation to calculate target with several different possible values between 0 and 1 for $\alpha$:

$$\hat{Y} = 10(\alpha \frac{\sum{} mandatory\:assignment\:marks}{number\:of\:mandatory\:assignments} + (1 - \alpha) \frac{\sum{} optional\:assignment\:marks}{number\:of\:optional\:assignments})$$

We will now calculate our results for final marks using the same value used by the authors of the R. Gonzalez paper:

$\alpha = 0.5$

In order to do that, we will start by normalizing all grades relative to their max possible grade to a 0 to be on a 0 to 1 scale.

Then, we will compute the final mark, which will allow us to compute the targets for our classifiers.

In [40]:
#hwe will start by defining a function that will assist us in dealing with the multiindex

def flattenHierarchicalCol(col,sep = '_'):
    '''converts multiindex columns into single index columns while retaining the hierarchical components'''
    if not type(col) is tuple:
        return col
    else:
        new_col = ''
        for leveli,level in enumerate(col):
            if not level == '':
                if not leveli == 0:
                    new_col += sep
                new_col += level
        return new_col

In [41]:
#step 1: current grade scales are varying between 0.7 and 1001. The fastest way to account for this 
support_table['assignment_mark'] = support_table['finalgrade'] / support_table['rawgrademax']

#step 2: For every student and every course, we can obtain the sum and number of both mandatory and optional assignments:
grade_estimation = support_table.groupby(['courseid', 'userid', 'mandatory_status']).agg({
                                                                                    'assignment_mark' : ['sum', 'count'],
                                                                                        })

#applies the function that removes multiindex
grade_estimation.columns = grade_estimation.columns.map(flattenHierarchicalCol)
grade_estimation.reset_index(inplace = True)

#now we can create an optional and a mandatory sum column for each sum 
grade_estimation['Optional'] = 0.5 * np.where(grade_estimation['mandatory_status'] == 0, #1 - alpha = 0.5,  
                                              grade_estimation['assignment_mark_sum'] / grade_estimation['assignment_mark_count'],
                                             np.nan)

grade_estimation['Mandatory'] = 0.5 * np.where(grade_estimation['mandatory_status'] == 1, # alpha = 0.5,  
                                              grade_estimation['assignment_mark_sum'] / grade_estimation['assignment_mark_count'],
                                              np.nan)

#for all intents and purposes, we can now remove the columns assignment_mark_sum
grade_estimation.drop(['assignment_mark_sum', 'assignment_mark_count'], axis = 1, inplace = True)

#we can now create a new pivot_table that perfectly arranges our intended result
targets_table = grade_estimation.pivot_table(index=['courseid','userid'], 
                                         columns=['mandatory_status'],
                                         values=['Optional', 'Mandatory'],aggfunc='sum')

#next we remove the columns that we do not want to keep, final result being: for each discipline, the optional and the mandatory grades
targets_table.columns.set_levels(['optional','mandatory'],level=1,inplace=True)
targets_table.columns = targets_table.columns.map(flattenHierarchicalCol)
targets_table.drop(['Mandatory_optional', 'Optional_mandatory'], axis = 1, inplace = True)

#next, we sum the columns and multiply by 10:
targets_table['final_mark'] = 10 * (targets_table['Mandatory_mandatory'] + targets_table['Optional_optional'].fillna(0))

#before finishing this cell, we will now drop the unncecessary columns and keep the final mark
targets_table.drop(['Mandatory_mandatory', 'Optional_optional'], axis = 1, inplace = True)
targets_table.reset_index(inplace = True)
del grade_estimation

We started the last cell with the normalization of the mark of each assignment. 

The cell finishes with a dataframe containing each curricular unit, each student attending it and the final mark of the student according to the formula we had placed previously.

In [42]:
targets_table.describe(include = 'all')

,courseid,userid,final_mark
count,30520,30520,26690.000000
unique,732,14121,NaN
top,2271.0,45851.0,NaN
freq,667,10,NaN
mean,NaN,NaN,3.414201
std,NaN,NaN,1.879728
min,NaN,NaN,0.000000
25%,NaN,NaN,2.394416
50%,NaN,NaN,3.597460
75%,NaN,NaN,4.500000


We now have 2 distinct table that will be invaluable for our work in future notebooks.

targets_table has stored every final mark obtain by each student attending the different courses of the university:

- From final_mark, we finally are able to calculate our target variables:
    - We can label students as at-risk or as overachievers depending on their mark,


- From support_table, we will need more robust sets of information to be used for feature extraction and engineering:
    - The startdate of each course,
    - The individual mark of each assignment and at what time the assignment was delivered,
    - The mandatory status of an assignment and whether it was or not delivered by the student in question.

In [43]:
support_table.describe(include = 'all', datetime_is_numeric = True)

,assign_id,courseid,startdate,userid,finalgrade,rawgrademax,sup_time,mandatory_status,delivered,assignment_mark
count,221523,221523,221523,221523,221523.000000,221523.000000,221523,221523.000000,221523.000000,221523.000000
unique,6347,732,NaN,14121,NaN,NaN,NaN,NaN,NaN,NaN
top,98648.0,2271.0,NaN,68626.0,NaN,NaN,NaN,NaN,NaN,NaN
freq,657,17871,NaN,131,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,2014-11-24 06:10:46.405113600,NaN,36.890038,107.643393,2015-02-09 13:41:47.534499840,0.563689,0.614713,0.416805
min,NaN,NaN,2014-07-31 22:00:00,NaN,0.000000,0.070000,2014-08-01 14:37:32,0.000000,0.000000,0.000000
25%,NaN,NaN,2014-09-09 22:00:00,NaN,0.000000,10.000000,2014-12-05 10:45:46.500000,0.000000,0.000000,0.000000
50%,NaN,NaN,2014-11-30 23:00:00,NaN,2.500000,41.000000,2015-02-13 12:45:39,1.000000,1.000000,0.368421
75%,NaN,NaN,2015-01-26 23:00:00,NaN,25.000000,100.000000,2015-04-20 19:53:53.500000,1.000000,1.000000,0.833334
max,NaN,NaN,2015-07-04 22:00:00,NaN,1001.000000,1001.000000,2015-07-28 21:45:39,1.000000,1.000000,1.000000


In [44]:
#save tables 
targets_table.to_csv('../Data/Modeling Stage/R_Gonz_targets_table.csv') 

support_table.drop(['finalgrade', 'rawgrademax'], axis = 1).to_csv('../Data/R_Gonz_support_table.csv')

#### Done for now

In notebook 2.2. we will rely on the activity logs and our support table to perform the necessary filtering and preprocessing of the data in order to make it compliant with our necessities. 